In [1]:
def calculate_angle(a, b, c):
    """Calculates the angle between three 2D points (in degrees)"""
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle


In [1]:


def calculate_angle(a, b, c):
    a = np.array(a)  #first
    b = np.array(b)  #mid
    c = np.array(c)  #end

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import threading

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def display_video(image):
    cv2.imshow('Mediapipe Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        return False
    return True

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Check if pose was detected
        if results.pose_landmarks is not None:
            # Extract landmarks
            landmarks = [(lmk.x, lmk.y) for lmk in results.pose_landmarks.landmark]

            # Calculate angle
            if len(landmarks) >= 3:
                shoulder, elbow, wrist = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value], landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
                angle = calculate_angle(shoulder, elbow, wrist)
                cv2.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Draw landmarks
            for landmark in landmarks:
                cx, cy = int(landmark[0] * frame.shape[1]), int(landmark[1] * frame.shape[0])
                cv2.circle(image, (cx, cy), 5, (0, 0, 255), -1)

        # Display video feed
        if not display_video(image):
            break

    cap.release()
    cv2.destroyAllWindows()



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
